### NOTE:
*In order to ease your transition to python and can use all the goodies out of the box, you will find every library you may need right from one import. You can access vectorbt's and mlfinlab's packages through the genie package or by importing them directly.*
  >e.g.:
  > from genie import vbt, mlf
  >e.g.:
  > from genie import *
  >e.g.:
  > import genie
  > genie.vbt.<obj>(...)
  >e.g.:
  > import genie
  > genie.<obj>(...)

  *When it comes to mlfinlab is preferably that you use it through the genie package. Overtime the benefits will pile up
  for debugging.*
  >e.g.:
  > import genie
  > genie.<obj>(...)
  *otherwise*
  >e.g.:
  > import genie
  > genie.mlf(...)

 *The same applies to many of the other packages that are imported in this file*
  >e.g.:
  > from genie import np # numpy
  > however you have to call them directly not through the genie package
  >e.g.:
  > * Not this
  > import genie
  > genie.array(...)

  > * Do this instead
  > import genie
  > genie.np.array(...)

  > or

  > * import numpy as np
  > np.array(...)

*This was done to avoid confusion between libraries*
*I will use all these import methods interchangably but just for demostration purposes lol, *



In [ ]:
import genie  # noqa: F401

# Introduction to VBT


Prior to doing any complex example or start to bring in custom functions or mlfinlab's goodies, lets use the classic moving average cross strategy to demostrate how easily it is to work with vbt. For most use cases vbt has all the tools you may need out of the box. For customers if we want to build them exactly what they desire we typically do need custom solutions, however that's because they don't make use of all the right tools at their disposal, like we can; lets make it work for us. Along the way we'll emphasize vbt's strengths and weaknesses, and you will notice me introducing custom functions that were built upon vbt in the next section.

Lets begin!

"""Example 1: Quick VBT Overview"""


In [ ]:
from genie import vbt

data = vbt.CSVData.fetch('Datas/tick_data.csv', index_col=0, parse_dates=True, infer_datetime_format=True)

print(data.wrapper.columns)  # Index(['Price', 'Volume'], dtype='object')
print(data.wrapper.prettify())  # Index(['Price', 'Volume'], dtype='object')
print(data)

In [ ]:
price = data.get('Price')
volume = data.get('Volume')
print(price.head())
print(f"Im Pretty: {vbt.prettify(price)}")

The vbt's prettify allows you to see the most useful components of the data type you are working with, of course making it look pretty ;) Perfect for adding vbt as needed throughout the pipeline and for use in notebooks like this one (brilliant idea alert!)

You can also access the wrapper's contents


In [ ]:
print(data.wrapper.index)
print(f"You're Pretty: {vbt.prettify(data.wrapper.index)}")

You get the point for this first example. Let's now go crazy and split the price series data into n_windows = 10 windows, each with n_bars = 10 using vbt's range splitter *(in most cases the processing of the data should occur before this step)*

In [ ]:
n_windows = 10
n_bars = 20

price_split, range_index = price.vbt.range_split(n=n_windows, range_len=n_bars)
print(vbt.prettify(price_split))
print(vbt.prettify(range_index))

We can see that the tick data was split into a pandas dataframe of length 20 and 10 columns, each representing a split in the data. However, do notice that the index now is not a datetime, rather, to save memory since our strategy will not deal with timeframe resampling it uses a type ```<class 'pandas.core.indexes.range.RangeIndex'>``` defined through ```RangeIndex(start=0, stop=20, step=1)```

In [ ]:
print(price_split.head())

It might feel like the data might get hard to work with from now on, however vbt has already made numba compiled function that can help in most cases, on top of that indicators can adjust their functionality according to the dimensions of the array, knowing in which axis you are working on is key, luckily we can take care of that prior to splitting the price into windows if need be.

To get the moving averages working we can make use of vbt's already available numba compiled indicator choices, as well as the out-of-the-box support for ```@talib``` and ```ta``` extensive, hyper-fast technical analysis functions (*C) along with access to WorldQuant's 101 Formulaic Alphas through the expression parser IndicatorFactory.from_wqa101. **more on these later**

In [ ]:
fast_ma = vbt.MA.run(close=price, window=20).ma # Notice we are using the price dataframe prior to split
slow_ma = vbt.MA.run(close=price, window=50).ma

print(fast_ma.head()) # the head will have nan's since we have no indicator warm up period
print(slow_ma.tail())

In [ ]:
#   The same can and is done when working with split windows
fast_ma = vbt.MA.run(close=price_split, window=20).ma # Using price_split instead of price dataframe (df)
slow_ma = vbt.MA.run(close=price_split, window=50).ma

print(fast_ma.head())
print(slow_ma.tail())

In [ ]:
#   and when using more than one window per indicator

# Arrays of moving average windows e.g. [10, 30, 50, ... ]
fast_ma_windows=genie.np.linspace(start=5, stop=45, num=5, dtype=int) # define how many elements
slow_ma_windows=genie.np.arange(50, 100, step=10, dtype=int) # or define the step between them

fast_ma = vbt.MA.run(close=price_split, window=fast_ma_windows).ma # Using price_split instead of price
slow_ma = vbt.MA.run(close=price_split, window=slow_ma_windows).ma

print(fast_ma.tail())
print(vbt.prettify(fast_ma))
print(vbt.prettify(fast_ma_windows))

Recall we had 10 splits each with 20 bars, if now for example the fast_ma has 5 windows now the indicator is still of length 20, and using multiple levels to define each column's key of the dataframe we have 50 columns, 5 moving average windows per every split

In [ ]:
entries = fast_ma.vbt.crossed_above(slow_ma) # all elements being compared with the nan's will result in False
exits = fast_ma.vbt.crossed_below(slow_ma)

print(entries.head())
print(entries.tail())

Because you are used to programing already, handling the multi-dimentional array I expect to be relatively easy once you get to practice hands on, yet is really the main source of error I've found in the vbt's community among newcomers, however this is where a flexible backend shines and where genie takes most of its inspiration.

For this simple case we can use the powerful vbt.Portfolio, which is a regular Python class subclassing Analyzable and having a range of Numba-compiled functions at its disposal. It's built similarly to other analyzable classes in the way that it has diverse class methods for instantiation from a range of inputs (such as Portfolio.from_signals taking signals), it's a state-full class capable of wrapping and indexing any Pandas-like objects contained inside it, and it can compute metrics and display (sub-)plots for quick introspection of the stored data.

Let's see it!

In [ ]:
# Runs entries/exit signals, allowing both long and short positions and reversing the position when appropriate
#   We'll cover how to customize this sophisticated method down this tutorial
pf = vbt.Portfolio.from_signals(close=price_split,
                                entries=entries, exits=exits,
                                direction=vbt.pf_enums.Direction.Both)
print(pf.prettify())

This portfolio obj *(pf)* contains all the information you need to reconstruct every strategy, parameter, window, signal without reconstructing all of it thus conserving memory and allows transitions down the genie pipeline. There are performance optimization steps to keep in mind but no need to worry about those for now.

Let's plot the default graphics at pf's disposal *(can only plot a single column if used like this, but it's not a problem since we definitely don't want to plot 1 million columns, and we have some workaround)*

In [ ]:
# first split and first moving average combination (column = 0)
pf.plot(column=0).show()

In [ ]:
pf.returns_stats(column=0) # will complain because the frequency that the price (tick data in this case) comes in to calculate some metrics. We could easily resample to lets say to minute bars.

In [ ]:
pf.returns_stats(agg_func=None  )

In [ ]:
from genie import np
pf.stats( agg_func=None).replace([np.inf, -np.inf], np.nan, inplace=False)

We can make it easier for ourselves (like genie does) and write a routine to plot our data.

In [ ]:
# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

In [ ]:


# Plot Main Graph______________________________________________________________
fig = price.plot()
fig = fast_ma.vbt.plot(fig=fig, trace_kwargs=dict(name="Fast_MA"))
fig = slow_ma.vbt.plot(fig=fig, trace_kwargs=dict(name="Slow_MA"))
fig = entries.vbt.signals.plot_as_entries(price, fig=fig)
fig = exits.vbt.signals.plot_as_exits(price, fig=fig)
fig.show()

In [ ]:




def example_indicator_function(price, fast_ma, slow_ma):
    fast_ma = vbt.MA.run(close=price, window=fast_ma).ma
    slow_ma = vbt.MA.run(close=price, window=slow_ma).ma
    entries = fast_ma.vbt.crossed_above(slow_ma)
    exits = fast_ma.vbt.crossed_below(slow_ma)
    return entries, exits


example_indicator = vbt.IF(
    class_name="_example_class_",
    short_name="_short_name_",
    input_names=["price"],
    param_names=["fast_ma_window", "slow_ma_window"],
    output_names=["entries", "exits"]
).with_apply_func(
    apply_func=example_indicator_function,
    price=price,
    fast_ma_window=20,
    slow_ma_window=50)

print(example_indicator)


In [ ]:
ind_results = example_indicator.run(price=price,
                                    fast_ma_window=[20, 30, 40],
                                    slow_ma_window=[50, 60, 70],
                                    )

"""Downloading/Loading Data"""

Downloads the last N bars of timeframe X data from YFData or Binance (CCXTData library)
There are multiple methods of aquiring data, it can be downloaded or loaded from a file and the function you use
will depend on the data you are using. Let's start by downloading some data from yahoo finance.

In [ ]:
from genie.datetime import datetime, timedelta

DAYS_TO_DOWNLOAD = 1000
end_time = datetime.now()
start_time = end_time - timedelta(days=DAYS_TO_DOWNLOAD)
data = vbt.YFData.fetch(
    # ["BTC-USD", "ETH-USD", "XMR-USD", "ADA-USD"],
    # ["BTC-USD", "ETH-USD"],
    # "BTC-USD",
    "ETH-USD",
    start=start_time,
    end=end_time,
    timeframe="1d",
    missing_index='drop'  # Creates problems with missing the index
)
print(f"YFData: ")
print(data)
#

the same can be achieved by using the ccxt lib which gives you access to a lot more crypto data sources and brokers

In [ ]:
This
returns
a
vbt
data
object <class 'vectorbtpro.data.cus

In [ ]:

#
data = vbt.CCXTData.fetch(
    "ETHUSDT",
    exchange="binance",
    start=start_time,
    end=end_time,
    timeframe="1 day"
)
print(f"CCXTData: ")
print(data)
print(type(data))

You can also load the data

In [ ]:
data_file_names = [
    "dollar_bars.csv",
]

# Searches for the data file names of csv type within the given directories
data = Data_Manager.fetch_data(data_file_names=data_file_names,
                               data_file_dirs=[".", "Datas", "Datas/Sample-Data"])
#
# This returns a vbt data object <class 'vectorbtpro.data.custom.<source>'> which essensially wraps the pandas
#   dataframe introducing new functionality and all of these can be saved and loaded as followed:
data.save("example_data")
data = vbt.Data.load("example_data")
# > Loading once again a <class 'vectorbtpro.data.base.Data'> object
print(type(data))
print(data.wrapper.columns)

# > I will cover how to access the data in the next section however for now lets take a look at how to get the OHLC data
# Split the Data
try:
    open_data = data.get('Open')
    low_data = data.get('Low')
    high_data = data.get('High')
    close_data = data.get('Close')
except:
    open_data = data.get('open')
    low_data = data.get('low')
    high_data = data.get('high')
    close_data = data.get('close')

print(open_data.head())
